In [2]:
using DrWatson
@quickactivate "Research"

In [10]:
using ProgressMeter, CoupledHMC, VecTarget

@info "System" Threads.nthreads()

┌ Info: System
│   Threads.nthreads() = 10
└ @ Main In[10]:3


In [11]:
target = get_target(TwoDimGaussianMixtures())

(dim = 2, logdensity = VecTarget.var"#logdensity#17"{VecTarget.GaussianMixtures{Array{Float64,1},MLToolkit.DistributionsX.BroadcastedNormalStd{Float64,Array{Float64,2},Array{Float64,1}}}}(VecTarget.GaussianMixtures{Array{Float64,1},MLToolkit.DistributionsX.BroadcastedNormalStd{Float64,Array{Float64,2},Array{Float64,1}}}([0.25, 0.4, 0.35], MLToolkit.DistributionsX.BroadcastedNormalStd{Float64,Array{Float64,2},Array{Float64,1}}(
m: [-1.0 0.0 1.0; -1.0 0.0 1.0]
s: [0.25]
)
)), get_grad = VecTarget.var"#16#18"{VecTarget.var"#logdensity#17"{VecTarget.GaussianMixtures{Array{Float64,1},MLToolkit.DistributionsX.BroadcastedNormalStd{Float64,Array{Float64,2},Array{Float64,1}}}}}(VecTarget.var"#logdensity#17"{VecTarget.GaussianMixtures{Array{Float64,1},MLToolkit.DistributionsX.BroadcastedNormalStd{Float64,Array{Float64,2},Array{Float64,1}}}}(VecTarget.GaussianMixtures{Array{Float64,1},MLToolkit.DistributionsX.BroadcastedNormalStd{Float64,Array{Float64,2},Array{Float64,1}}}([0.25, 0.4, 0.35], MLTo

In [12]:
args = (
    n_mc = 500,
    n_samples_max = 100,
#     TS_list = [
#         EndPointTS,
#         MultinomialTS,
#         ContractiveMultinomialTS{OT},
#         ContractiveMultinomialTS{Pairing},
#     ],
#     ϵinc_list = [(ϵ, 10) for ϵ in collect(0.1:0.05:0.3)],
#     Linc_list = [(0.1, L) for L in collect(10:5:30)],
    γ = 1/20,
    σ = 1e-3,
)

@info "Args" args...


┌ Info: Args
│   n_mc = 500
│   n_samples_max = 100
│   γ = 0.05
│   σ = 0.001
└ @ Main In[12]:16


In [13]:
function run_gmm(TS, ϵ, L)
    alg = CoupledHMCSampler(rinit=rand, TS=TS, ϵ=ϵ, L=L, γ=args.γ, σ=args.σ)
    does_meets = Vector{Bool}(undef, args.n_mc)
    progress = Progress(args.n_mc)
    Threads.@threads for i in 1:args.n_mc
        samples = sample_until_meeting(target, alg; n_samples_max=args.n_samples_max)
        does_meets[i] = does_meet(samples[end])
        next!(progress)
    end 
    n_meeting = sum(does_meets)
    efficiency = round(n_meeting / args.n_mc; digits=3)
    @info "Efficiency: $efficiency"
    
#     wsave(projectdir("neurips_2020", "gmm.bson"), @dict(args, results))
end

run_gmm(EndPointTS, 0.1, 10)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:19
┌ Info: Efficiency: 0.636
└ @ Main In[13]:12
